In [37]:
import os
import shutil
import datetime
import subprocess
import re
import multiprocessing
import time
import json
import datetime

In [38]:
def get_curr_time():
    return datetime.datetime.now().strftime("%d-%h-%y | %H:%M")

In [39]:
def _load_seq_from_file(file_name):
    with open(file_name) as f:
        seq = f.readlines()
    return "".join([l.strip() for l in seq])

In [40]:

SIMULATION_ROOT_DIR = "simulation"
SIMULATION_EXEC = os.path.join(SIMULATION_ROOT_DIR, "bin")
SIMULATION_LIBS = os.path.join(SIMULATION_ROOT_DIR, "libs")
SIMULATION_RUN_DIRS= os.path.join(SIMULATION_ROOT_DIR, "runs")


In [41]:
def create_run_dir_name(gene_name):
    return gene_name + "_" + datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")

In [42]:
os.environ["PATH"] += os.pathsep + os.path.join(os.getcwd(), SIMULATION_EXEC)
os.environ["PATH"]

'/home/ubuntu/anaconda3/envs/lammps/bin:/home/ubuntu/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/ubuntu/CTP/simulation/bin:/home/ubuntu/CTP/simulation/bin'

In [43]:
def gen_sim_data(folder_name):
    seq = _load_seq_from_file(os.path.join(folder_name, "seq.txt"))
    seq_name = folder_name.split(os.sep)[-1]
    dirs = next(os.walk(os.path.join(folder_name, 'directCoexistence')))[1]
    dirs = [dir[4:] for dir in dirs]
    return {"gene_name": seq_name, "seq": seq, "temps": dirs}

In [44]:
def gen_sim_data_done(folder_name):
    seq = _load_seq_from_file(os.path.join(folder_name, "seq.txt"))
    seq_name = folder_name.split(os.sep)[-1]
    dirs = next(os.walk(folder_name))[1]
    dirs = [dir[4:] for dir in dirs]
    return {"gene_name": seq_name, "seq": seq, "temps": dirs}

In [45]:
def read_sim_data(seq_file='simulation/simulation_to_run.json'):
    with open(seq_file) as f:
        return json.load(f)

In [46]:
def run_sample_lmp(temp_dir_name, CPUs_per_process):
    f = open(os.path.join(temp_dir_name, "sample.log"), "w+")
    CPUs_per_process_arg = str(CPUs_per_process)
    subprocess.run(["mpiexec", "-np", CPUs_per_process_arg, "lmp", "-in", "sample.lmp"], stdout=f, cwd=temp_dir_name)
    f.close()

In [47]:
def run_simulation(gene_name, seq, temps,
                   CPUs,
                   CPUs_per_process, 
                   run_dirs=SIMULATION_RUN_DIRS,
                   simulation_dir=SIMULATION_ROOT_DIR,
                   delete_lammpstrj=True):
    
    if len(seq) < 40:
        CPUs_for_first_stages = 8
    else:
        CPUs_for_first_stages = 16
    
    processes =  CPUs // CPUs_per_process
    
    simulation_dir_name = create_run_dir_name(gene_name)
    simulation_dir_name = os.path.join(run_dirs, simulation_dir_name)
    print(simulation_dir_name)
    os.mkdir(simulation_dir_name)
   
    with open(os.path.join(simulation_dir_name, "seq.txt"), "w+") as f:
        for i, letter in enumerate(seq, start=1):
            if i < len(seq):
                f.write(letter+"\n")
            else:
                f.write(letter)
        f.truncate()
    shutil.copy(os.path.join(SIMULATION_LIBS, "seq2config.py"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_charges_one_letter.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_charges_three_letters.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_types_three_letters.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_types_one_letter.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "potentials.dat"), simulation_dir_name)

    shutil.copy(os.path.join(SIMULATION_LIBS, "inputSingleChain.lmp"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "compress.lmp"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "sample.lmp"), simulation_dir_name)
    
    
    
    subprocess.run(["python", "seq2config.py"], cwd=simulation_dir_name)
    os.rename(os.path.join(simulation_dir_name, "myconfig.dat"), os.path.join(simulation_dir_name, "initialChainConfig.dat"))

    # subprocess.run(["python", "seq2config.py"], cwd=simulation_dir_name)

    print("step 1: running input single chain")
    f = open(os.path.join(simulation_dir_name, "inputSingleChain.log"), "w+")
    subprocess.run(["mpiexec", "-np", str(CPUs_for_first_stages), "lmp", "-in", "inputSingleChain.lmp"], stdout=f, cwd=simulation_dir_name)
    f.close()
    os.rename(os.path.join(simulation_dir_name, "finalSingleChainStructure.dat"), os.path.join(simulation_dir_name, "singleChainConfig.dat"))

    print("step 2: running compress single chain")
    f = open(os.path.join(simulation_dir_name, "compress.log"), "w+")
    subprocess.run(["mpiexec", "-np", str(CPUs_for_first_stages), "lmp", "-in", "compress.lmp"], stdout=f, cwd=simulation_dir_name)
    f.close()
    os.rename(os.path.join(simulation_dir_name, "fusfinalStructureSlab.dat"), os.path.join(simulation_dir_name, "initialSlab.dat"))
    
    print("step 3: direct coexistence")
    with open(os.path.join(simulation_dir_name, "replicate.lammpstrj")) as f:
        replicate_data = f.readlines()
        data_to_copy = []
        for line in replicate_data[::-1]:
            if line == 'ITEM: ATOMS id mol type q xu yu zu\n':
                break
            else:
                data_to_copy.append(line)
        data_to_copy = [line for line in data_to_copy[::-1]]

    with open(os.path.join(simulation_dir_name, "initialSlab.dat")) as f:
        slab_data = f.read()

    data_to_swap = "".join(data_to_copy)

    slab_data = re.sub(r'(Atoms # full\n)([\n\s0-9-.e]*)', rf'\1\n{data_to_swap}', slab_data)
    slab_data = re.sub(r'Velocities([\n\s0-9-.e]*)', r'\n', slab_data)

    zlo_zhi = re.findall(r"\d.+\szlo zhi\n", slab_data)

    zlo, zhi = [float(v.strip()) for v in zlo_zhi[0].split(" ")[:2]]

    deltaZ = (zhi- zlo)*2

    zhi = zhi+deltaZ

    zlo = zlo - deltaZ

    zlo_zhi_r = f'\n{zlo} {zhi} zlo zhi'

    slab_data = re.sub(r'\n.*zlo zhi', zlo_zhi_r, slab_data)

    with open(os.path.join(simulation_dir_name, "initialSlab.dat"), 'w+') as f:
        f.write(slab_data)
    
   
    temp_buf = []
    for temp in sorted(temps):
        temp_dir_name = os.path.join(simulation_dir_name, str(temp))
        if not os.path.exists(temp_dir_name):
            os.mkdir(temp_dir_name)
        shutil.copy(os.path.join(SIMULATION_LIBS, "potentials.dat"), temp_dir_name)
        shutil.copy(os.path.join(SIMULATION_LIBS, "da.py"), temp_dir_name)
        shutil.copy(os.path.join(simulation_dir_name, "initialSlab.dat"), temp_dir_name)
        with open(os.path.join(SIMULATION_LIBS, "sample.lmp")) as f:
            sample_lmp = f.read()
        sample_lmp = re.sub(
           r"(variable temperature equal)(\s\d+)", 
           rf"\1 {temp}", 
           sample_lmp
        )
       
        with open(os.path.join(temp_dir_name, "sample.lmp"), "w+") as f:
           f.write(sample_lmp)

        temp_buf.append({"temp": temp, "dir": temp_dir_name})
        if len(temp_buf) == proccesses:
            print(f"running {temp_buf}")
            print(get_curr_time())
            start_time = time.time()
            with multiprocessing.Pool(len(temp_buf)) as pool:
                params = [(v["dir"],) for v in temp_buf]
                results = [pool.apply_async(run_sample_lmp, args = p + (CPUs_per_process,)) for p in params]
    
                for r in results:
                    r.get()
                    #print('\t', r.get())
            temp_buf = []
            print(f'execution time: {(time.time() - start_time) / (60*60)} hours')
            print(get_curr_time())
    
    if temp_buf:
        print(f"running {temp_buf}")
        print(f"remaining temp, CPUs_pp = {CPUs//len(temp_buf)}")
        print(get_curr_time())
        
        start_time = time.time()
        #with multiprocessing.Pool(CPUs_per_process) as pool:
        with multiprocessing.Pool(len(temp_buf)) as pool:
            params = [(v["dir"],) for v in temp_buf]
            results = [pool.apply_async(run_sample_lmp, args = p + (CPUs//len(temp_buf),)) for p in params]

            for r in results:
                r.get()
                # rint('\t', r.get())
        temp_buf = []
        print(f'execution time: {(time.time() - start_time) / (60*60)} hours')
        print(get_curr_time())


    if delete_lammpstrj:
        for temp_folder in next(os.walk(simulation_dir_name))[1]:
            for file in os.listdir(os.path.join(simulation_dir_name, temp_folder)):
                if file.endswith("lammpstrj"):
                    # print(os.path.join("simulation", "runs", r_folder, temp_folder, file))
                    os.remove(os.path.join(simulation_dir_name, temp_folder, file))
        for file in next(os.walk(simulation_dir_name))[2]:
            if file.endswith("lammpstrj"):
                # print(os.path.join("simulation", "runs", r_folder, temp_folder, file))
                os.remove(os.path.join(simulation_dir_name, file))
        
    try:    
        #Calculating ps densities
        print("calculating ps densities")
        if not os.path.exists(os.path.join(simulation_dir_name, "ps_densities")):
            os.mkdir(os.path.join(simulation_dir_name, "ps_densities"))
        
        shutil.copy(os.path.join(SIMULATION_LIBS, "ppd.py"), os.path.join(simulation_dir_name, "ps_densities"))
        for temp in next(os.walk(simulation_dir_name))[1]:
            if temp != "ps_densities":
                print(os.path.join(simulation_dir_name, str(temp)))
                subprocess.run(["python", "da.py", "densities_chunked2.dat"], cwd=os.path.join(simulation_dir_name, str(temp)))
                os.rename(os.path.join(simulation_dir_name, temp, "densities_chunked2.dat_concentrations.txt"),
                          os.path.join(simulation_dir_name, "ps_densities", f"densities_chunked_temp{temp}.txt"))
    
        subprocess.run(["python", "ppd.py"], cwd=os.path.join(simulation_dir_name, "ps_densities"))
    
    except Exception:
        pass
        
        # f = open(os.path.join(simulation_dir_name, "compress.log"), "w+")
        # subprocess.run(["mpiexec", "-np", f"CPUs_per_process", "lmp", "-in", "sample.lmp"], stdout=f, cwd=temp_dir_name)
        # f.close()


In [119]:
simulations_completed = ['EWSR1_-10G+10T',
                         'VPAP_HCMVA_P16790_Wildtype',
                         "SUPT6H(Q7KZ85)_Wildtype", 
                         "MCDB_SYNE7_Q8GJM6_Wildtype",
                         "LORF1_-5S+5T"]

In [13]:
def get_completed_simulations():
    sim_dirs = next(os.walk(os.path.join("simulation", "runs")))[1]
    comp_sims = ["_".join(dir_name.split("_")[:-6]) for dir_name in sim_dirs if dir_name!='']
    return [sim for sim in comp_sims if sim]

In [16]:
simulations_completed = get_completed_simulations()
simulations_completed

['TIA_-5w+5f_j',
 'HOME3_-4S+4T',
 'SUPT6H(Q7KZ85)_Wildtype',
 'LORF1_Q9UN81_Wildtype',
 'TIA_-5w+5f',
 'LORF1_-5S+5T',
 'TIA_-5w+5f_j',
 'VPAP_HCMVA_P16790_Wildtype',
 'CEP3_HHV11_P04289_Wildtype',
 'MCDB_SYNE7_Q8GJM6_Wildtype',
 'HOME3_Q9NSC5_Wildtype',
 'EWSR1_-10G+10T',
 'GAG_-5S+5T',
 'GAG_-5T+5G']

In [17]:
seq_to_run = read_sim_data()
seq_to_run

{'AVPGVG': {'seq': 'PAVPGVGAVPGVGAVPGVGAVPGVGG',
  'temps': [250,
   270,
   290,
   300,
   307,
   314,
   321,
   328,
   335,
   342,
   349,
   356,
   363,
   370]},
 'AVPGVGAVPGVGAVPGVGAVPGVGAVPGVGCVPGVG': {'seq': 'PAVPGVGAVPGVGAVPGVGAVPGVGAVPGVGCVPGVGAVPGVGAVPGVGAVPGVGAVPGVGAVPGVGCVPGVGAVPGVGAVPGVGAVPGVGAVPGVGAVPGVGCVPGVGAVPGVGAVPGVGAVPGVGAVPGVGAVPGVGCVPGVGG',
  'temps': [250,
   270,
   290,
   300,
   307,
   314,
   321,
   328,
   335,
   342,
   349,
   356,
   363,
   370]},
 'C7GS49_YEAS2_Wildtype': {'seq': 'MSFLSKLSQIRKSTTASKAQVQDPLPKKNDEEYSLLPKNYIRDEDPAVKRLKELRRQELLKNGALAKKSGVKRKRGTSSGSEKKKIERNDDDEGGIGIRFKRSIGASHAPLKPVVRKKPEPIKKMSFEELMKQAENNEKQPPKVKSSEPVTKERPHFNKPGFKSSKRPQKKASPGATLRGVSSGGNSIKSSDSPKPVKLNLSTNGFAQPNRRLKEKLESRKQKSRYQDDYDEEDNDMDDFIEDDEDEGYHSKSKHSNGPGYDRDEIWAMFNRGKKRSEYDYDELE',
  'temps': [250,
   270,
   290,
   300,
   307,
   314,
   321,
   328,
   335,
   342,
   349,
   356,
   363,
   370]},
 'CHKB_Q9Y259_Wildtype': {'seq': 'MAAEATAVAGSGAVGGCLAKDGLQQS

In [23]:
julias_simulations = ["TDP", "TIA", "EWSR", "RBM", "FUS"]

In [24]:
for k in seq_to_run:
    for j in julias_simulations:
        if k.lower().startswith(j.lower()):
            print(k)

EWSR1_-10G+10T
EWSR1_-10Q+10N
EWSR1_-10S+10t
EWSR1_-10T+10G
EWSR1_-10t+10s
EWSR1_-10Y
EWSR1_-20G+20T
EWSR1_-20Q+20N
EWSR1_-20S+20T
EWSR1_-20T+20G
EWSR1_-20T+20S
EWSR1_-20Y
EWSR1_-20Y+20F
EWSR1_-30S+30T
EWSR1_-30T+30G
EWSR1_-30T+30S
EWSR1_-30Y+30F
EWSR1_-4N+4Q
EWSR1_-5Q+5N
EWSR1_-5Y
EWSR1_-5Y+5F
fus_-10g+10t
fus_-10q+10n
fus_-10s+10t
fus_-10t+10g
fus_-10t+10s
fus_-14y
fus_-14y+14f
fus_-20q+20n
fus_-28y+28f
fus_-5q+5n
fus_-5t+5g
fus_-5t+5s
fus_-7y
fus_-7y+7f
RBM14_-10T+10G
RBM14_-10T+10S
RBM14_-10Y
RBM14_-20Q+20N
RBM14_-20Y+20F
RBM14_-30Y+30F
RBM14_-5Q+5N
RBM14_-5T+5G
RBM14_-5T+5S
RBM14_-5Y
RBM14_-5Y+5F
TIA_-4y+4f
TIA_-4Y+4W
TIA_-5w+5f
TIA_-5W+5Y
TIA_-9N+9Q
TIA_-9y+9f
TIA_-9y+9w


In [25]:
len(seq_to_run["TIA_-5w+5f"]["seq"])

97

In [26]:
class JuliasExeption(Exception):
    pass

In [27]:
seq_to_run['TIA_-5w+5f']

{'seq': 'MINPVQQQNQIGYPQPYGQFGQFYGNAQQIGQYMPNGFQVPAYGMYGQAFNQQGFNQTQSSAPFMGPNYGVQPPQGQNGSMLPNQPSGYRVAGYETN',
 'temps': [250,
  270,
  290,
  300,
  307,
  314,
  321,
  328,
  335,
  342,
  349,
  356,
  363,
  370]}

In [29]:
seq_to_run = {**seq_to_run, **{'TIA_-5w+5f_j': {'seq': 'MINPVQQQNQIGYPQPYGQFGQFYGNAQQIGQYMPNGFQVPAYGMYGQAFNQQGFNQTQSSAPFMGPNYGVQPPQGQNGSMLPNQPSGYRVAGYETN',
 'temps': [
  334,
  335,
  337,
  339,
  341,
  342
  ]}}}

In [30]:
seq_to_run["TIA_-5w+5f_j"]

{'seq': 'MINPVQQQNQIGYPQPYGQFGQFYGNAQQIGQYMPNGFQVPAYGMYGQAFNQQGFNQTQSSAPFMGPNYGVQPPQGQNGSMLPNQPSGYRVAGYETN',
 'temps': [334, 335, 337, 339, 341, 342]}

In [32]:
CPUs=96
CPUs_per_process = 16
proccesses = CPUs // CPUs_per_process 

#seq_to_run = read_sim_data()

log_name = f'simulation_run_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")}.log'

with open(os.path.join("simulation", "logs", log_name), "w+") as f:
    print(get_curr_time())
    mesg = f"CPUs {CPUs}, proccesses {proccesses}, CPUs_per_process {CPUs_per_process}\n"
    print(mesg)
    f.writelines(mesg+"\n")
    
    # for seq_name in seq_to_run:
    for sim_number, seq_name in enumerate(["TIA_-5w+5f_j"]):
        # if seq_name in simulations_completed:
        #     print(f"Sequence was already calculated {seq_name}")
        #     continue


        if seq_name in ["TIA_-4y+4f", "TIA_-5w+5f", "TIA_-5w+5f_j"]:
            print(f"replicating Julia's {seq_name}")
        
     
        mesg = f"running simulation for {seq_name}"
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
        
        start_time = time.time()
        
        sim_args = {"gene_name": seq_name,
                    "seq": seq_to_run[seq_name]['seq'],
                    "temps": seq_to_run[seq_name]['temps']}
          
        mesg = f"seq len {len(sim_args['seq'])}"
        print(mesg)
        f.writelines(mesg+"\n")
    
        mesg = f"total temps {len(sim_args['temps'])}"
        print(mesg)
        f.writelines(mesg+"\n")
        
        run_simulation(** sim_args | {"CPUs": CPUs, "CPUs_per_process": CPUs_per_process})
        # for i in range(111111111):
        #     pass
        mesg = f'execution time: {(time.time() - start_time) / (60*60)} hours'
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
    
        mesg = f'\n'
        print(mesg)
        f.writelines(mesg+"\n")
        #break

#f.close()

15-Aug-24 | 07:55
CPUs 96, proccesses 6, CPUs_per_process 16

replicating Julia's TIA_-5w+5f_j
running simulation for TIA_-5w+5f_j
15-Aug-24 | 07:55
seq len 97
total temps 6
simulation/runs/TIA_-5w+5f_j_2024_08_15_07_08_1723708550
step 1: running input single chain


/home/ubuntu/CTP/simulation/runs/TIA_-5w+5f_j_2024_08_15_07_08_1723708550/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


KeyboardInterrupt: 

In [55]:
seq_to_run[seq_name]

KeyError: 'TIA_-5w+5f_j'

In [ ]:
for temp in next(os.walk(simulation_dir_name))[1]:
    if temp != "ps_densities" and temp != ".ipynb_checkpoints":
        print(os.path.join(simulation_dir_name, str(temp)))
        subprocess.run(["python", "da.py", "densities_chunked2.dat"], cwd=os.path.join(simulation_dir_name, str(temp)))
        os.rename(os.path.join(simulation_dir_name, temp, "densities_chunked2.dat_concentrations.txt"),
                  os.path.join(simulation_dir_name, "ps_densities", f"densities_chunked_temp{temp}.txt"))

In [41]:
simulation_dir_name = "simulation/runs/TIA_-5w+5f_2024_08_13_11_08_1723548979/"
subprocess.run(["python", "ppd.py"], cwd=os.path.join(simulation_dir_name, "ps_densities"))

/home/ubuntu/anaconda3/envs/lammps/lib/python3.9/site-packages/matplotlib/collections.py:197: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)
/home/ubuntu/anaconda3/envs/lammps/lib/python3.9/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/ubuntu/anaconda3/envs/lammps/lib/python3.9/site-packages/matplotlib/cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


File densities_chunked_temp250.txt read successfully with 2 lines.
Original data lines:
Dense phase concentration: 0.4254039915444544 +/- 0.3534728172056548
Dilute phase concentration: 9.94670557332115e-07 +/- 3.060325839163841e-06
Filtered data lines: 2
Dense phase concentration: 0.4254039915444544 +/- 0.3534728172056548
Dilute phase concentration: 9.94670557332115e-07 +/- 3.060325839163841e-06
File densities_chunked_temp270.txt read successfully with 2 lines.
Original data lines:
Dense phase concentration: 0.384236177936489 +/- 0.32023249501611495
Dilute phase concentration: 6.386061623909017e-07 +/- 2.0135341940972123e-06
Filtered data lines: 2
Dense phase concentration: 0.384236177936489 +/- 0.32023249501611495
Dilute phase concentration: 6.386061623909017e-07 +/- 2.0135341940972123e-06
File densities_chunked_temp290.txt read successfully with 2 lines.
Original data lines:
Dense phase concentration: 0.34013213970206746 +/- 0.284517170231312
Dilute phase concentration: 0.00044724940

CompletedProcess(args=['python', 'ppd.py'], returncode=0)

In [36]:
CPUs=96
CPUs_per_process = 16
proccesses = CPUs // CPUs_per_process 

seq_to_run = read_sim_data()

log_name = f'simulation_run_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")}.log'

with open(os.path.join("simulation", "logs", log_name), "w+") as f:
    print(get_curr_time())
    mesg = f"CPUs {CPUs}, proccesses {proccesses}, CPUs_per_process {CPUs_per_process}\n"
    print(mesg)
    f.writelines(mesg+"\n")
    
    # for seq_name in seq_to_run:
    for sim_number, seq_name in enumerate(sorted(seq_to_run, key=lambda x: len(seq_to_run[x]['seq']))):
        if seq_name in simulations_completed:
            print(f"Sequence was already calculated {seq_name}")
            continue
        
        try:
            for j in julias_simulations:
                if seq_name.lower().startswith(j.lower()):
                    print(f"Julias sequence {seq_name}")
                    raise JuliasExeption
        except JuliasExeption:
            continue
        
     
        mesg = f"running simulation for {seq_name}"
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
        
        start_time = time.time()
        
        sim_args = {"gene_name": seq_name,
                    "seq": seq_to_run[seq_name]['seq'],
                    "temps": seq_to_run[seq_name]['temps']}
          
        mesg = f"seq len {len(sim_args['seq'])}"
        print(mesg)
        f.writelines(mesg+"\n")
    
        mesg = f"total temps {len(sim_args['temps'])}"
        print(mesg)
        f.writelines(mesg+"\n")
        
        run_simulation(** sim_args | {"CPUs": CPUs, "CPUs_per_process": CPUs_per_process})
        # for i in range(111111111):
        #     pass
        mesg = f'execution time: {(time.time() - start_time) / (60*60)} hours'
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
    
        mesg = f'\n'
        print(mesg)
        f.writelines(mesg+"\n")
        #break

#f.close()

15-Aug-24 | 08:00
CPUs 96, proccesses 6, CPUs_per_process 16

running simulation for VAPGVG
15-Aug-24 | 08:00
seq len 20
total temps 14
simulation/runs/VAPGVG_2024_08_15_08_08_1723708840
step 1: running input single chain


/home/ubuntu/CTP/simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': 250, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/250'}, {'temp': 270, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/270'}, {'temp': 290, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/290'}, {'temp': 300, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/300'}, {'temp': 307, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/307'}, {'temp': 314, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/314'}]
15-Aug-24 | 08:01
execution time: 0.21795631196763782 hours
15-Aug-24 | 08:14
running [{'temp': 321, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/321'}, {'temp': 328, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/328'}, {'temp': 335, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/335'}, {'temp': 342, 'dir': 'simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/342'}, {'temp': 349, 'dir': 'simulation/runs/V

/home/ubuntu/CTP/simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/300/da.py:62: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(super_gaussian, coords, avg_profile, p0=initial_guess, maxfev=20000)


Dense phase concentration: 0.16615966099071208 +/- 0.0019675670703119207
Dilute phase concentration: 0.1663607105263158 +/- 0.003168314213478888
simulation/runs/VAPGVG_2024_08_15_08_08_1723708840/314


...

/home/ubuntu/CTP/simulation/runs/I0FUN3_MACMU_-5Q+5N_2024_08_17_09_08_1723887509/270/da.py:62: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(super_gaussian, coords, avg_profile, p0=initial_guess, maxfev=20000)


Dense phase concentration: 0.4324521652451128 +/- 0.3572717366800208
Dilute phase concentration: 1.1149126794258374e-06 +/- 4.783187400719532e-06
File densities_chunked_temp250.txt read successfully with 2 lines.
Original data lines:
Dense phase concentration: 0.4657186565091094 +/- 0.3843330413629753
Dilute phase concentration: 0.0 +/- 0.0
Filtered data lines: 2
Dense phase concentration: 0.4657186565091094 +/- 0.3843330413629753
Dilute phase concentration: 0.0 +/- 0.0
File densities_chunked_temp270.txt read successfully with 2 lines.
Original data lines:
Dense phase concentration: 0.4324521652451128 +/- 0.3572717366800208
Dilute phase concentration: 1.1149126794258374e-06 +/- 4.783187400719532e-06
Filtered data lines: 2
Dense phase concentration: 0.4324521652451128 +/- 0.3572717366800208
Dilute phase concentration: 1.1149126794258374e-06 +/- 4.783187400719532e-06
File densities_chunked_temp290.txt read successfully with 2 lines.
Original data lines:
Dense phase concentration: 0.40362

/home/ubuntu/CTP/simulation/runs/I0FUN3_MACMU_Wildtype_2024_08_17_12_08_1723897245/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 1: running input single chain
step 2: running compress single chain
step 3: direct coexistence
running [{'temp': 250, 'dir': 'simulation/runs/I0FUN3_MACMU_Wildtype_2024_08_17_12_08_1723897245/250'}, {'temp': 270, 'dir': 'simulation/runs/I0FUN3_MACMU_Wildtype_2024_08_17_12_08_1723897245/270'}, {'temp': 290, 'dir': 'simulation/runs/I0FUN3_MACMU_Wildtype_2024_08_17_12_08_1723897245/290'}, {'temp': 300, 'dir': 'simulation/runs/I0FUN3_MACMU_Wildtype_2024_08_17_12_08_1723897245/300'}, {'temp': 307, 'dir': 'simulation/runs/I0FUN3_MACMU_Wildtype_2024_08_17_12_08_1723897245/307'}, {'temp': 314, 'dir': 'simulation/runs/I0FUN3_MACMU_Wildtype_2024_08_17_12_08_1723897245/314'}]
17-Aug-24 | 12:23



KeyboardInterrupt



In [123]:

seq_to_run_test = {"CEP3_HHV11_P04289_Wildtype": {'seq': 'MGLSFSGARPCCCRNNVLITDDGEVVSLTAHDFDVVDIESEEEGNFYVPPDMRGVTRAPGRQRLRSSDPPSRHTHRRTPGGACPATQFPPPMSDSE',
                                                 'temps': [180, 190, 200, 210, 220]}}

In [139]:
CPUs=96
CPUs_per_process = 16
proccesses = CPUs // CPUs_per_process 

seq_to_run = read_sim_data()

log_name = f'simulation_run_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")}.log'

with open(os.path.join("simulation", "logs", log_name), "w+") as f:
    print(get_curr_time())
    mesg = f"CPUs {CPUs}, proccesses {proccesses}, CPUs_per_process {CPUs_per_process}\n"
    print(mesg)
    f.writelines(mesg+"\n")
    
    # for seq_name in seq_to_run:
    for seq_name in sorted(seq_to_run_test, key=lambda x: len(seq_to_run_test[x]['seq'])):
        if seq_name in simulations_completed:
            print(f"Sequence was already calculated {seq_name}")
            continue
        mesg = f"running simulation for {seq_name}"
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
        
        start_time = time.time()
        
        sim_args = {"gene_name": seq_name,
                    "seq": seq_to_run_test[seq_name]['seq'],
                    "temps": seq_to_run_test[seq_name]['temps']}
          
        mesg = f"seq len {len(sim_args['seq'])}"
        print(mesg)
        f.writelines(mesg+"\n")
    
        mesg = f"total temps {len(sim_args['temps'])}"
        print(mesg)
        f.writelines(mesg+"\n")
        
        run_simulation(** sim_args | {"CPUs": CPUs, "CPUs_per_process": CPUs_per_process})
        # for i in range(111111111):
        #     pass
        mesg = f'execution time: {(time.time() - start_time) / (60*60)} hours'
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
    
        mesg = f'\n'
        print(mesg)
        f.writelines(mesg+"\n")
        #break

#f.close()

11-Aug-24 | 15:45
CPUs 96, proccesses 6, CPUs_per_process 16

Sequence was already calculated CEP3_HHV11_P04289_Wildtype


In [140]:

CPUs=96
CPUs_per_process = 16
proccesses = CPUs // CPUs_per_process 

seq_to_run = read_sim_data()

log_name = f'simulation_run_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")}.log'

with open(os.path.join("simulation", "logs", log_name), "w+") as f:
    print(get_curr_time())
    mesg = f"CPUs {CPUs}, proccesses {proccesses}, CPUs_per_process {CPUs_per_process}\n"
    print(mesg)
    f.writelines(mesg+"\n")
    
    # for seq_name in seq_to_run:
    for seq_name in sorted(seq_to_run, key=lambda x: len(seq_to_run[x]['seq'])):
        if seq_name in simulations_completed:
            print(f"Sequence was already calculated {seq_name}")
            continue
        mesg = f"running simulation for {seq_name}"
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
        
        start_time = time.time()
        
        sim_args = {"gene_name": seq_name,
                    "seq": seq_to_run[seq_name]['seq'],
                    "temps": seq_to_run[seq_name]['temps']}
          
        mesg = f"seq len {len(sim_args['seq'])}"
        print(mesg)
        f.writelines(mesg+"\n")
    
        mesg = f"total temps {len(sim_args['temps'])}"
        print(mesg)
        f.writelines(mesg+"\n")
        
        run_simulation(** sim_args | {"CPUs": CPUs, "CPUs_per_process": CPUs_per_process})
        # for i in range(111111111):
        #     pass
        mesg = f'execution time: {(time.time() - start_time) / (60*60)} hours'
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
    
        mesg = f'\n'
        print(mesg)
        f.writelines(mesg+"\n")
        #break

#f.close()

11-Aug-24 | 15:46
CPUs 96, proccesses 6, CPUs_per_process 16

Sequence was already calculated MCDB_SYNE7_Q8GJM6_Wildtype
Sequence was already calculated SUPT6H(Q7KZ85)_Wildtype
Sequence was already calculated LORF1_-5S+5T
Sequence was already calculated LORF1_Q9UN81_Wildtype
Sequence was already calculated HOME3_-4S+4T
running simulation for HOME3_Q9NSC5_Wildtype
11-Aug-24 | 15:46
seq len 48
total temps 14
simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176
step 1: running input single chain


/home/ubuntu/CTP/simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': 250, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/250'}, {'temp': 270, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/270'}, {'temp': 290, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/290'}, {'temp': 300, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/300'}, {'temp': 307, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/307'}, {'temp': 314, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/314'}]
11-Aug-24 | 15:54
execution time: 3.202032267053922 hours
11-Aug-24 | 19:06
running [{'temp': 321, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/321'}, {'temp': 328, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/328'}, {'temp': 335, 'dir': 'simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723

/home/ubuntu/CTP/simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/300/da.py:62: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(super_gaussian, coords, avg_profile, p0=initial_guess, maxfev=20000)


Dense phase concentration: 0.20839993038723023 +/- 0.0011550685182438267
Dilute phase concentration: 0.20768634265075375 +/- 0.001317377572444679
simulation/runs/HOME3_Q9NSC5_Wildtype_2024_08_11_15_08_1723391176/314


...

Dense phase concentration: 0.22925758675731597 +/- 0.001701456750290393
Dilute phase concentration: 0.2298198300879397 +/- 0.001123719584880388
simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/363


/home/ubuntu/CTP/simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/363/da.py:62: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(super_gaussian, coords, avg_profile, p0=initial_guess, maxfev=20000)


Dense phase concentration: 0.22953072376588826 +/- 0.0014097269670059922
Dilute phase concentration: 0.22923942462311558 +/- 0.0014742425424563064
simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/250


Traceback (most recent call last):
  File "/home/ubuntu/CTP/simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/250/da.py", line 146, in <module>
    main()
  File "/home/ubuntu/CTP/simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/250/da.py", line 133, in main
    dense, dilute, dense_stdev, dilute_stdev = process_density_file(file_name)
  File "/home/ubuntu/CTP/simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/250/da.py", line 121, in process_density_file
    dense, dilute, dense_stdev, dilute_stdev = get_dilute_and_dense_concentrations(density, zPosition)
  File "/home/ubuntu/CTP/simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/250/da.py", line 99, in get_dilute_and_dense_concentrations
    left_interface_coord, right_interface_coord = super_gaussian_fd_method(coords, density)
  File "/home/ubuntu/CTP/simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/250/da.py", line 62, in super_gaussian_fd_method
    popt, _ = curve_fit(super_gaussian, coords, avg_profile, 

FileNotFoundError: [Errno 2] No such file or directory: 'simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/250/densities_chunked2.dat_concentrations.txt' -> 'simulation/runs/GAG_-5T+5G_2024_08_12_12_08_1723464030/ps_densities/densities_chunked_temp250.txt'

In [98]:
sim_args

{'gene_name': 'LORF1_-5S+5T',
 'seq': 'MGKKQNRKTGNTKTQTASPPPKERTTSPATEQTWM',
 'temps': [250,
  270,
  290,
  300,
  307,
  314,
  321,
  328,
  335,
  342,
  349,
  356,
  363,
  370]}

In [99]:
gene_name = sim_args['gene_name']
seq = sim_args['seq']
temps = sim_args['temps']
                   
run_dirs=SIMULATION_RUN_DIRS
                   
simulation_dir=SIMULATION_ROOT_DIR
                  
delete_lammpstrj=True



processes =  CPUs // CPUs_per_process

simulation_dir_name = create_run_dir_name(gene_name)
simulation_dir_name = os.path.join(run_dirs, simulation_dir_name)
print(simulation_dir_name)
os.mkdir(simulation_dir_name)

with open(os.path.join(simulation_dir_name, "seq.txt"), "w+") as f:
    for i, letter in enumerate(seq, start=1):
        if i < len(seq):
            f.write(letter+"\n")
        else:
            f.write(letter)
    f.truncate()
shutil.copy(os.path.join(SIMULATION_LIBS, "seq2config.py"), simulation_dir_name)
shutil.copy(os.path.join(SIMULATION_LIBS, "aa_charges_one_letter.txt"), simulation_dir_name)
shutil.copy(os.path.join(SIMULATION_LIBS, "aa_charges_three_letters.txt"), simulation_dir_name)
shutil.copy(os.path.join(SIMULATION_LIBS, "aa_types_three_letters.txt"), simulation_dir_name)
shutil.copy(os.path.join(SIMULATION_LIBS, "aa_types_one_letter.txt"), simulation_dir_name)
shutil.copy(os.path.join(SIMULATION_LIBS, "potentials.dat"), simulation_dir_name)

shutil.copy(os.path.join(SIMULATION_LIBS, "inputSingleChain.lmp"), simulation_dir_name)
shutil.copy(os.path.join(SIMULATION_LIBS, "compress.lmp"), simulation_dir_name)
shutil.copy(os.path.join(SIMULATION_LIBS, "sample.lmp"), simulation_dir_name)



subprocess.run(["python", "seq2config.py"], cwd=simulation_dir_name)


os.rename(os.path.join(simulation_dir_name, "myconfig.dat"), os.path.join(simulation_dir_name, "initialChainConfig.dat"))

subprocess.run(["python", "seq2config.py"], cwd=simulation_dir_name)

print("step 1: running input single chain")
f = open(os.path.join(simulation_dir_name, "inputSingleChain.log"), "w+")
subprocess.run(["lmp", "-in", "inputSingleChain.lmp"], stdout=f, cwd=simulation_dir_name)
f.close()
os.rename(os.path.join(simulation_dir_name, "finalSingleChainStructure.dat"), os.path.join(simulation_dir_name, "singleChainConfig.dat"))

print("step 2: running compress single chain")
f = open(os.path.join(simulation_dir_name, "compress.log"), "w+")
subprocess.run(["lmp", "-in", "compress.lmp"], stdout=f, cwd=simulation_dir_name)
f.close()
os.rename(os.path.join(simulation_dir_name, "fusfinalStructureSlab.dat"), os.path.join(simulation_dir_name, "initialSlab.dat"))

print("step 3: direct coexistence")
with open(os.path.join(simulation_dir_name, "replicate.lammpstrj")) as f:
    replicate_data = f.readlines()
    data_to_copy = []
    for line in replicate_data[::-1]:
        if line == 'ITEM: ATOMS id mol type q xu yu zu\n':
            break
        else:
            data_to_copy.append(line)
    data_to_copy = [line for line in data_to_copy[::-1]]

with open(os.path.join(simulation_dir_name, "initialSlab.dat")) as f:
    slab_data = f.read()

data_to_swap = "".join(data_to_copy)

slab_data = re.sub(r'(Atoms # full\n)([\n\s0-9-.e]*)', rf'\1\n{data_to_swap}', slab_data)
slab_data = re.sub(r'Velocities([\n\s0-9-.e]*)', r'\n', slab_data)

zlo_zhi = re.findall(r"\d.+\szlo zhi\n", slab_data)

zlo, zhi = [float(v.strip()) for v in zlo_zhi[0].split(" ")[:2]]

deltaZ = (zhi- zlo)*2

zhi = zhi+deltaZ

zlo = zlo - deltaZ

zlo_zhi_r = f'\n{zlo} {zhi} zlo zhi'

slab_data = re.sub(r'\n.*zlo zhi', zlo_zhi_r, slab_data)

with open(os.path.join(simulation_dir_name, "initialSlab.dat"), 'w+') as f:
    f.write(slab_data)


temp_buf = []
for temp in sorted(temps):
    temp_dir_name = os.path.join(simulation_dir_name, str(temp))
    if not os.path.exists(temp_dir_name):
        os.mkdir(temp_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "potentials.dat"), temp_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "da.py"), temp_dir_name)
    shutil.copy(os.path.join(simulation_dir_name, "initialSlab.dat"), temp_dir_name)
    with open(os.path.join(SIMULATION_LIBS, "sample.lmp")) as f:
        sample_lmp = f.read()
    sample_lmp = re.sub(
       r"(variable temperature equal)(\s\d+)", 
       rf"\1 {temp}", 
       sample_lmp
    )
   
    with open(os.path.join(temp_dir_name, "sample.lmp"), "w+") as f:
       f.write(sample_lmp)

    temp_buf.append({"temp": temp, "dir": temp_dir_name})
    if len(temp_buf) == proccesses:
        print(f"running {temp_buf}")
        print(get_curr_time())
        start_time = time.time()
        with multiprocessing.Pool(CPUs_per_process) as pool:
            params = [(v["dir"],) for v in temp_buf]
            results = [pool.apply_async(run_sample_lmp, args = (p, CPUs_per_process)) for p in params]

            for r in results:
                print('\t', r.get())
        temp_buf = []
        print(f'execution time: {(time.time() - start_time) / (60*60)} hours')
        print(get_curr_time())

if temp_buf:
    print(f"running {temp_buf}")
    print(get_curr_time())
    start_time = time.time()
    #with multiprocessing.Pool(CPUs_per_process) as pool:
    with multiprocessing.Pool(CPUs//len(temp_buf)) as pool:
        params = [(v["dir"],) for v in temp_buf]
        results = [pool.apply_async(run_sample_lmp, args = p + (CPUs_per_process,)) for p in params]

        for r in results:
            print('\t', r.get())
    temp_buf = []
    print(f'execution time: {(time.time() - start_time) / (60*60)} hours')
    print(get_curr_time())

simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252
step 1: running input single chain


/home/ubuntu/CTP/simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])
/home/ubuntu/CTP/simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': 250, 'dir': 'simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/250'}, {'temp': 270, 'dir': 'simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/270'}, {'temp': 290, 'dir': 'simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/290'}, {'temp': 300, 'dir': 'simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/300'}, {'temp': 307, 'dir': 'simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/307'}, {'temp': 314, 'dir': 'simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/314'}]
10-Aug-24 | 10:44


TypeError: expected str, bytes or os.PathLike object, not tuple

In [ ]:
temp_dir_name, CPUs_per_process

In [106]:
def run_sample_lmp(temp_dir_name, CPUs_per_process):
    f = open(os.path.join(temp_dir_name, "sample.log"), "w+")
    CPUs_per_process_arg = str(CPUs_per_process)
    subprocess.run(["mpiexec", "-np", CPUs_per_process_arg, "lmp", "-in", "sample.lmp"], stdout=f, cwd=temp_dir_name)
    f.close()

In [109]:
params

('simulation/runs/LORF1_-5S+5T_2024_08_10_10_08_1723286252/250', 'z')

In [110]:
with multiprocessing.Pool(CPUs_per_process) as pool:
            params = [(v["dir"],) for v in temp_buf]
            results = [pool.apply_async(run_sample_lmp, p+ (CPUs_per_process,)) for p in params]

            for r in results:
                print('\t', r.get())

	 None
	 None
	 None
	 None
	 None
	 None


In [ ]:
simulation_names = ["VPAP_HCMVA_P16790_Wildtype", "VP22_HHV11_P10233_Wildtype"]

In [17]:
simulation_names = ["VP22_HHV11_P10233_Wildtype"]

In [33]:

CPUs=32
proccesses = 2
CPUs_per_process = CPUs // proccesses 

f = open(os.path.join("simulation", "logs", f'simulation_run_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")}.log'), "w+")

mesg = f"CPUs {CPUs}, proccesses {proccesses}, CPUs_per_process, {CPUs_per_process}\n"
print(mesg)
f.writelines(mesg+"\n")

for sim_name in simulation_names:
    mesg = f"running simulation for {sim_name}"
    print(mesg)
    f.writelines(mesg+"\n")
    
    start_time = time.time()
    sim_args = gen_sim_data(os.path.join("simulation", "simulation_data_temp", sim_name))
    
    mesg = f"seq len {len(sim_args['seq'])}"
    print(mesg)
    f.writelines(mesg+"\n")

    mesg = f"total temps {len(sim_args['temps'])}"
    print(mesg)
    f.writelines(mesg+"\n")
    
    run_simulation(**sim_args)
    # for i in range(111111111):
    #     pass
    mesg = f'execution time: {(time.time() - start_time) / (60*60)} minutes'
    print(mesg)
    f.writelines(mesg+"\n")

    mesg = f'\n'
    print(mesg)
    f.writelines(mesg+"\n")

f.close()

CPUs 32, proccesses 2, CPUs_per_process, 16

running simulation for VPAP_HCMVA_P16790_Wildtype
seq len 161
total temps 14
simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114
step 1: running input single chain


/home/ubuntu/CTP/simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])
/home/ubuntu/CTP/simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': '250', 'dir': 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/250'}, {'temp': '270', 'dir': 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/270'}]
	 None
	 None
execution time: 7.061490030818516 hours
running [{'temp': '290', 'dir': 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/290'}, {'temp': '300', 'dir': 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/300'}]
	 None
	 None
execution time: 6.668494150373671 hours
running [{'temp': '307', 'dir': 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/307'}, {'temp': '314', 'dir': 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/314'}]
	 None
	 None
execution time: 6.674376530117459 hours
running [{'temp': '321', 'dir': 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/321'}, {'temp': '328', 'dir': 

/home/ubuntu/CTP/simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/300/da.py:62: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(super_gaussian, coords, avg_profile, p0=initial_guess, maxfev=20000)


Dense phase concentration: 0.22563537865799588 +/- 0.0019390040151780678
Dilute phase concentration: 0.22218646576633166 +/- 0.001328993204467062
simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/314


/home/ubuntu/CTP/simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/314/da.py:62: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(super_gaussian, coords, avg_profile, p0=initial_guess, maxfev=20000)


Dense phase concentration: 0.22492335205438957 +/- 0.0018368902440444998
Dilute phase concentration: 0.2236995358040201 +/- 0.0017861328328657663
simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/.ipynb_checkpoints


python: can't open file '/home/ubuntu/CTP/simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/.ipynb_checkpoints/da.py': [Errno 2] No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/.ipynb_checkpoints/densities_chunked2.dat_concentrations.txt' -> 'simulation/runs/VPAP_HCMVA_P16790_Wildtype_2024_08_01_11_08_1722512114/ps_densities/densities_chunked_temp.ipynb_checkpoints.txt'

In [379]:
gen_sim_data_done("simulation/simulation_data_temp/VPAP_HCMVA_P16790_Wildtype")

{'gene_name': 'VPAP_HCMVA_P16790_Wildtype',
 'seq': 'QRGDPFDKNYVGNSGKSRGGGGGGGSLSSLANAGGLHDDGPGLDNDLMNEPMGLGGLGGGGGGGGKKHDRGGGGGSGTRKMSSGGGGGDHDHGLSSKEKYEQHKITSYLTSKGGSGGGGGGGGGGLDRNSGNYFNDAKEESDSEDSVTFEFVPNTKKQKCG',
 'temps': ['ctCoexistence', 'icateChain']}

In [410]:
gen_sim_data_done("simulation/simulation_data_temp/VP22_HHV11_P10233_Wildtype")

{'gene_name': 'VP22_HHV11_P10233_Wildtype',
 'seq': 'MTSRRSVKSGPREVPRDEYEDLYYTPSSGMASPDSPPDTSRRGALQTRSRQRGEVRFVQYDESDYALYGGSSSEDDEHPEVPRTRRPVSGAVLSGPGPARAPPPPAGSGGAGRTPTTAPRAPRTQRVATKAPAAPAAETTRGRKSAQPESAALPDAPASTAPTRSKTPAQGLARKLHFSTAPPNPDAP',
 'temps': ['ctCoexistence', 'icateChain']}

In [382]:
gen_sim_data("simulation/simulation_data_temp/VPAP_HCMVA_P16790_Wildtype")

{'gene_name': 'VPAP_HCMVA_P16790_Wildtype',
 'seq': 'QRGDPFDKNYVGNSGKSRGGGGGGGSLSSLANAGGLHDDGPGLDNDLMNEPMGLGGLGGGGGGGGKKHDRGGGGGSGTRKMSSGGGGGDHDHGLSSKEKYEQHKITSYLTSKGGSGGGGGGGGGGLDRNSGNYFNDAKEESDSEDSVTFEFVPNTKKQKCG',
 'temps': ['321',
  '363',
  '314',
  '290',
  '342',
  '356',
  '300',
  '335',
  '270',
  '370',
  '250',
  '328',
  '307',
  '349']}

In [373]:
run_simulation(**gen_sim_data_done("simulation/simulation_data_temp/CCMM_SYNE7_-10s+10t"))

simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353
step 1: running input single chain


/home/ubuntu/CTP/simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])
/home/ubuntu/CTP/simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': '150', 'dir': 'simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/150'}, {'temp': '170', 'dir': 'simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/170'}]
	 None
	 None
running [{'temp': '190', 'dir': 'simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/190'}, {'temp': '200', 'dir': 'simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/200'}]
	 None
	 None
running [{'temp': '205', 'dir': 'simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/205'}, {'temp': '210', 'dir': 'simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/210'}]
	 None
	 None
running [{'temp': '215', 'dir': 'simulation/runs/CCMM_SYNE7_-10s+10t_2024_07_27_16_07_1722097353/215'}]
	 None
calculating ps densities


FileNotFoundError: [Errno 2] No such file or directory: 'simulation/libs/pdd.py'

In [354]:
run_simulation(**gen_sim_data("simulation/simulation_data_temp/UBQL2_-4Q+4N"))

simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318
step 1: running input single chain


/home/ubuntu/CTP/simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])
/home/ubuntu/CTP/simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': '250', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/250'}, {'temp': '270', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/270'}]
	 None
	 None
running [{'temp': '290', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/290'}, {'temp': '300', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/300'}]
	 None
	 None
running [{'temp': '307', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/307'}, {'temp': '314', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/314'}]
	 None
	 None
running [{'temp': '321', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/321'}, {'temp': '328', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/328'}]
	 None
	 None
running [{'temp': '335', 'dir': 'simulation/runs/UBQL2_-4Q+4N_2024_07_26_20_07_1722026318/335'}, {'temp': '342', 'dir': 'simulation/runs/UB


KeyboardInterrupt

